In [2]:
from pathlib import Path
from pandas.core.interchange.dataframe_protocol import DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import re
import pandas as pd
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame, Series
import numpy as np
from sklearn.cluster import DBSCAN
from umap import UMAP
from sklearn.manifold import trustworthiness
import optuna
import featuretools as ft
from woodwork.logical_types import Double, Datetime, Integer, Categorical

In [17]:
df = pd.read_pickle("../data/edited/full_df_rdy_1.pkl").reset_index(drop=True)
df["index"] = df.index

In [8]:
df.tail()

,index,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,...,Which of the following best describes your work position?_DevOps,Which of the following best describes your work position?_back,Which of the following best describes your work position?_front,Which of the following best describes your work position?_lead,Which of the following best describes your work position?_other,Which of the following best describes your work position?_support,Why or why not?,Why_or_why_not_1_Cluster,Why_or_why_not_Cluster,Why or why not?.1
1425,1425,0.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1426,1426,0.0,3.0,1.0,1.0,3.0,3.0,3.0,1.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
1427,1427,0.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1430,1430,0.0,3.0,1.0,3.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1431,1431,0.0,3.0,0.0,3.0,0.0,3.0,0.0,0.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df.shape

(1011, 95)

In [19]:
es = ft.EntitySet(id="single_table")

es = es.add_dataframe(dataframe_name="data", dataframe=df, index="index")

In [22]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="data",
    agg_primitives=[],  # keine Aggregationen (weil keine zweite Tabelle)
    trans_primitives=["add_numeric", "and", "or"],
    max_depth=2  # Tiefe der Featurekombination
)

C:\Users\achim\Desktop\OSMI_Mental_Health\venv\Lib\site-packages\featuretools\synthesis\deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(
C:\Users\achim\Desktop\OSMI_Mental_Health\venv\Lib\site-packages\featuretools\synthesis\dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  trans_primitives: ['and', 'or']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [27]:
feature_matrix.shape

(1011, 4465)

In [28]:
print(f"{feature_matrix.shape[1] - df.shape[1]} Spalten durch Featuretools hinzugefügt")

4370 Spalten durch Featuretools hinzugefügt


In [31]:
pd.to_pickle(feature_matrix, "df_mit_featuretools.pkl")